# Pregunta 8
Realizar una left join para incluir a todos los empleados incluso a aquellos sin departamentos.<br>

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.appName('Preguntas Join').getOrCreate()
df = spark.read.csv('./datasets/employees.csv', header=True, inferSchema=True)

In [28]:
# Creamos df con Department y dept_id -> Sales:1, HR:None, IT:3 y que ya sea null para HR en Department
df_employees = df.withColumn('dept_id',
                              F.when(F.col('Department') == 'Sales', 1)
                              .when(F.col('Department') == 'HR', F.lit(None))
                              .when(F.col('Department') == 'IT', 3))\
                                .drop('Department')
# Creamos df solo con dept_id y Department
df_department = df.select('Department').distinct()\
                .withColumn('dept_id',
                                F.when(F.col('Department') == 'Sales', 1)
                                .when(F.col('Department') == 'HR', 2)
                                .when(F.col('Department') == 'IT', 3))

df_employees.show(6) 

+---+------------------+---+------+------+------------+-----------------+----------+--------+-----------+-------+-------+
| ID|              Name|Age|Gender|Salary|Joining Date|Performance Score|Experience|  Status|   Location|Session|dept_id|
+---+------------------+---+------+------+------------+-----------------+----------+--------+-----------+-------+-------+
|  1|      Cory Escobar| 48|Female|  5641|  2015-05-03|              2.0|        16|  Active|   New York|  Night|   NULL|
|  2|   Timothy Sanchez| 25| Other|  4249|  2020-11-09|              2.0|        11|Inactive|Los Angeles|Evening|      1|
|  3|      Chad Nichols| 57| Other|  3058|  2019-02-12|             NULL|         1|Inactive|   New York|Morning|      1|
|  4|Christine Williams| 58|Female|  5895|  2017-09-08|              2.0|        13|Inactive|Los Angeles|Evening|      3|
|  5|      Amber Harris| 35| Other|  4317|  2020-02-15|              5.0|        16|Inactive|   New York|Evening|      3|
|  6|       Ashley Howe|

In [30]:
# left join para incluir todos los empleados
df_join = df_employees.join(df_department, on='dept_id', how='left')
df_join.show()  


+-------+---+------------------+---+------+------+------------+-----------------+----------+--------+-----------+-------+----------+
|dept_id| ID|              Name|Age|Gender|Salary|Joining Date|Performance Score|Experience|  Status|   Location|Session|Department|
+-------+---+------------------+---+------+------+------------+-----------------+----------+--------+-----------+-------+----------+
|   NULL|  1|      Cory Escobar| 48|Female|  5641|  2015-05-03|              2.0|        16|  Active|   New York|  Night|      NULL|
|      1|  2|   Timothy Sanchez| 25| Other|  4249|  2020-11-09|              2.0|        11|Inactive|Los Angeles|Evening|     Sales|
|      1|  3|      Chad Nichols| 57| Other|  3058|  2019-02-12|             NULL|         1|Inactive|   New York|Morning|     Sales|
|      3|  4|Christine Williams| 58|Female|  5895|  2017-09-08|              2.0|        13|Inactive|Los Angeles|Evening|        IT|
|      3|  5|      Amber Harris| 35| Other|  4317|  2020-02-15|      